<h1 align="center">Training Custom NER Model Using Flair</h1>

Data scientist.: Dr.Eddy Giusepe Chirinos Isidro

Começamos estudando o modelo treinado em `Conll-03` (`NER de 4 classes`). Para mais detalhes pode seguir o seguinte tutorial:

* [Treinando NER customizado](https://medium.com/thecyphy/training-custom-ner-model-using-flair-df1f9ea9c762)

In [7]:
import warnings
warnings.filterwarnings('ignore')

## [Flair](https://github.com/flairNLP/flair)

In [14]:
from flair.models import SequenceTagger
from flair.data import Sentence

# Carregamos o Modelo
tagger = SequenceTagger.load('ner')

sentence = Sentence('George Washington went to Washington state and Francisco Pizarro went to Perú.')

# Prever tags NER
tagger.predict(sentence)

# Print da sentence com a tag predita
print(sentence.to_tagged_string())

for entity in sentence.get_spans('ner'):
    print(entity)


2022-12-14 23:11:22,675 loading file /home/eddygiusepe/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-12-14 23:11:25,000 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
Sentence: "George Washington went to Washington state and Francisco Pizarro went to Perú ." → ["George Washington"/PER, "Washington"/LOC, "Francisco Pizarro"/PER, "Perú"/LOC]
Span[0:2]: "George Washington" → PER (0.9993)
Span[4:5]: "Washington" → LOC (0.9614)
Span[7:9]: "Francisco Pizarro" → PER (0.9996)
Span[11:12]: "Perú" → LOC (0.9942)


Além disso, podemos obter uma `pontuação de confiança` de cada uma das entidades previstas:

In [15]:
print(sentence.to_dict(tag_type='ner'))

{'text': 'George Washington went to Washington state and Francisco Pizarro went to Perú.', 'ner': [{'value': 'PER', 'confidence': 0.9993289113044739}, {'value': 'LOC', 'confidence': 0.9613563418388367}, {'value': 'PER', 'confidence': 0.999558836221695}, {'value': 'LOC', 'confidence': 0.994195282459259}]}


## Criação de nossos Dados

Mas por que `treinar` os próprios `taggers` de sequência? Em minha carreira profissional relativamente jovem, me deparei com problemas onde havia algumas frases/sequência textual, a serem extraídas de um documento inteiro. 

`Por exemplo:` 

considere um contrato, não apenas qualquer tipo de contrato, mas um contrato de arrendamento. Nesses contratos, as `“entidades”` como o nome das partes envolvidas, a data de rescisão, etc., tornam as informações de interesse. Agora, imagine o cenário em que esses contratos `não são estruturados`. Se eles fossem estruturados, poderíamos apenas extrair as referidas entidades escrevendo algumas regras básicas.


Agora, para criar um modelo que possa encontrar essas entidades a partir do texto fornecido, primeiro precisamos criar um `corpus de treinamento`. 


Vamos dar uma olhada rápida em como esse corpus é lido.

```
from flair.data import Corpus
from flair.datasets import ColumnCorpus


# Definimos as colunas
columns = {0: 'text', 1: 'ner'}

# Esta é a pasta onde residem os arquivos train, dev e test
data_folder = '/home/eddygiusepe/Imagens/Eddy_codigos/NLP_Transformers/Training_custom_NER/data'

# Inicie um corpus usando formato de coluna, pasta de dados e os nomes dos arquivos train, dev e test
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt') 

```                              

In [18]:
import pandas as pd

data = pd.DataFrame([["Horses are too tall and they pretend to care about your feelings", [("Horses", "ANIMAL")]],
                  ["Who is Shaka Khan?", [("Shaka Khan", "PERSON")]],
                  ["I like London and Berlin.", [("London", "LOCATION"), ("Berlin", "LOCATION")]],
                  ["There is a banyan tree in the courtyard", [("banyan tree", "TREE")]]], columns=['text', 'annotation'])

data.head()                  

,text,annotation
0,Horses are too tall and they pretend to care a...,"[(Horses, ANIMAL)]"
1,Who is Shaka Khan?,"[(Shaka Khan, PERSON)]"
2,I like London and Berlin.,"[(London, LOCATION), (Berlin, LOCATION)]"
3,There is a banyan tree in the courtyard,"[(banyan tree, TREE)]"


## Lendo o corpus

Finalmente, estamos prontos para carregar o corpus que criamos e começar com o treinamento. Vamos começar carregando nosso corpus.

In [32]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus


# Definimos as colunas
columns = {0: 'text', 1: 'ner'}

# Esta é a pasta onde residem os arquivos train, dev e test
data_folder = '/home/eddygiusepe/Imagens/Eddy_codigos/NLP_Transformers/Training_custom_NER/data'

# Inicie um corpus usando formato de coluna, pasta de dados e os nomes dos arquivos train, dev e test
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt') 

2022-12-15 00:45:07,410 Reading data from /home/eddygiusepe/Imagens/Eddy_codigos/NLP_Transformers/Training_custom_NER/data
2022-12-15 00:45:07,412 Train: /home/eddygiusepe/Imagens/Eddy_codigos/NLP_Transformers/Training_custom_NER/data/train.txt
2022-12-15 00:45:07,414 Dev: /home/eddygiusepe/Imagens/Eddy_codigos/NLP_Transformers/Training_custom_NER/data/dev.txt
2022-12-15 00:45:07,415 Test: /home/eddygiusepe/Imagens/Eddy_codigos/NLP_Transformers/Training_custom_NER/data/test.txt


Agora que carregamos nosso corpus, podemos usar este objeto corpus para obter suas informações como:

In [36]:
print(len(corpus.train))


4


In [37]:
print(corpus.train[0].to_tagged_string('ner'))

Sentence: "Horses are too tall and they pretend to care about your feelings" → ["Horses"/ANIMAL]


# Training 

Continuando, a próxima coisa é definir a `tag` que queremos que nosso modelo seja capaz de prever e criar o dicionário de tags, que é apenas todos os rótulos disponíveis no corpus.

In [50]:
# tag para prever
tag_type = 'ner'

# Fazer dicionário de tags a partir do corpus
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

2022-12-15 01:53:07,371 Computing label dictionary. Progress:


4it [00:00, 7172.82it/s]

2022-12-15 01:53:07,441 Dictionary created for label 'ner' with 5 values: LOCATION (seen 2 times), ANIMAL (seen 1 times), PERSON (seen 1 times), TREE (seen 1 times)


A próxima coisa é cuidar das `Embeddings`. A beleza do talento está em tudo o que ele permite que você faça com os `Embeddings`. Você pode escolher entre vários modelos pré-treinados para criar `Embeddings`, até mesmo empilhar os referidos `Embeddings` de talento com `BERT`, `ELMO` e outros enfeites poderosos usando a classe `StackedEmbedding`. E obviamente, treine seus próprios `Embeddings`. Ainda dentro do escopo deste blog, vamos avançar com o treinamento. Os detalhes dos `Embeddings` são maravilhosamente documentados [aqui](https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_3_WORD_EMBEDDING.md).

In [51]:
from flair.embeddings import WordEmbeddings, StackedEmbeddings, TokenEmbeddings
from typing import List


embedding_types : List[TokenEmbeddings] = [
        WordEmbeddings('glove'),
        ## other embeddings
        ]
embeddings : StackedEmbeddings = StackedEmbeddings(
                                 embeddings=embedding_types)

O próximo passo é inicializar o `Sequence Tagger`. Conceitualmente falando, o que é treinado no back-end é uma `LSTM bidirecional`. `Flair` permite que você passe uma `flag` (sinalizador) para usar os [campos aleatórios condicionais](https://en.wikipedia.org/wiki/Conditional_random_field) também. Vamos definir o referido tagger e ver a arquitetura também.

In [53]:
from flair.models import SequenceTagger

tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True) # CRF pode levar em consideração o contexto

print(tagger)

2022-12-15 01:53:31,946 SequenceTagger predicts: Dictionary with 17 tags: O, S-LOCATION, B-LOCATION, E-LOCATION, I-LOCATION, S-ANIMAL, B-ANIMAL, E-ANIMAL, I-ANIMAL, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-TREE, B-TREE, E-TREE, I-TREE
SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=19, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)


`Flair` ainda não deixou de ser incrível, tudo o que resta fazer agora é escrever exatamente mais três linhas de código para criar mágica.

In [ ]:
from flair.trainers import ModelTrainer


trainer : ModelTrainer = ModelTrainer(tagger, corpus)

trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)


`Isso define nosso modelo de treinamento`. Agora, há algumas coisas a serem observadas. Lembre-se de que, ao criar o objeto de corpus, passamos na validação e nos dados de teste ali mesmo? Isso porque o `flair` internamente faz muitas coisas por você, durante o treino e até mesmo após o treino. Ele cria um novo diretório chamado `resources` em seu diretório de trabalho atual, onde você encontrará tudo, desde os `logs de treinamento`, `informações da Loss` até as `previsões no conjunto de testes` com uma pontuação de confiança. Sob o mesmo diretório em `'resources/taggers/example-ner'` nosso modelo será salvo.

<font color="orange">Por fim, temos um modelo treinado e agora podemos `usá-lo para prever as tags` de uma nova sequência de texto. Isso novamente pode ser feito em algumas linhas mostradas no trecho a seguir.</font>

In [76]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load the trained model
model = SequenceTagger.load('/home/eddygiusepe/Imagens/Eddy_codigos/NLP_Transformers/Training_custom_NER/resources/taggers/example-ner/final-model.pt')

# create example sentence
sentence = Sentence('I life in london!')

# predict the tags
model.predict(sentence)
print(sentence.to_tagged_string())

2022-12-15 02:08:25,193 loading file /home/eddygiusepe/Imagens/Eddy_codigos/NLP_Transformers/Training_custom_NER/resources/taggers/example-ner/final-model.pt
2022-12-15 02:08:25,727 SequenceTagger predicts: Dictionary with 19 tags: O, S-LOCATION, B-LOCATION, E-LOCATION, I-LOCATION, S-ANIMAL, B-ANIMAL, E-ANIMAL, I-ANIMAL, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-TREE, B-TREE, E-TREE, I-TREE, <START>, <STOP>
Sentence: "I life in london !" → ["london"/LOCATION]
